In [12]:
import numpy as np
import pandas as pd
import scipy.optimize
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [13]:
def agonist_only_p_active_theory(c_agonist, log_Kd_active, log_Kd_inactive, beta_deltaE):
    "theoretical curve for mglur5 active probability"

    Kd_active = 10**(log_Kd_active)
    Kd_inactive = 10**(log_Kd_inactive)
    a = (1 + c_agonist/Kd_active)

    b = (1 + c_agonist/Kd_inactive)

    return a/(a+b*np.exp(-beta_deltaE))

def resid(param, c, p_active):
    return p_active - agonist_only_p_active_theory(c, *param)



In [14]:
#set up plot
from bokeh.plotting import curdoc 
curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[Ligands]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

colors=bokeh.palettes.Magma[11]

ligand_smooth=np.logspace(-13,-5,200)

for i,deltaE in enumerate(np.linspace(-1,-4,9)):    
    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, -10.9, -9.9, deltaE), color=colors[i],legend_label='delta E='+str(deltaE))

p.legend.location = "top_left"

bokeh.io.show(p);

In [15]:
#set up plot
from bokeh.plotting import curdoc 
curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[Ligands]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

colors=bokeh.palettes.Magma[11]
ligand_smooth=np.logspace(-13,-5,200)

for i,Kd_inactive in enumerate(np.linspace(-10,-8,9)):    
    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, -10.9, Kd_inactive, -1), color=colors[i],legend_label='Kd_inactive='+str(Kd_inactive))

p.legend.location = "top_left"

bokeh.io.show(p);

In [16]:
#set up plot
from bokeh.plotting import curdoc 
curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[Ligands]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

colors=bokeh.palettes.Magma[11]
ligand_smooth=np.logspace(-13,-5,200)

for i,Kd_active in enumerate(np.linspace(-11,-8,9)):    
    p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, Kd_active, -9, -1), color=colors[i],legend_label='Kd_active='+str(Kd_active))

p.legend.location = "top_left"

bokeh.io.show(p);

In [20]:
from natsort import natsorted
import glob

In [22]:
fnames = natsorted(glob.glob("./jiawei_trupath_csv/MOR_*_Gi1_normalize_deer+Gi.xlsx"))
fnames

['./jiawei_trupath_csv/MOR_BU72_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_DAMGO_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_MP_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_PZM21_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_TRV130_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_buprenorphine_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_lofentanil_Gi1_normalize_deer+Gi.xlsx',
 './jiawei_trupath_csv/MOR_morphine_Gi1_normalize_deer+Gi.xlsx']

In [23]:
df = pd.read_excel(fnames[0])
df


,conc,BU72
0,1.000000e-05,0.758763
1,1.000000e-06,0.633927
2,1.000000e-07,0.862248
3,1.000000e-08,0.619476
4,1.000000e-09,0.567454
...,...,...
91,1.000000e-08,0.866254
92,1.000000e-09,0.711398
93,1.000000e-10,0.566599
94,1.000000e-11,0.348001


In [7]:
curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[Ligands] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')
colors=bokeh.palettes.d3['Category10'][10]

for i,ligand in enumerate(df.columns[1:]):
    c, p_active = df['conc'].values, df[ligand].values
    p.circle(c, p_active, size=7, color=colors[i],legend_label=ligand)

bokeh.io.show(p);

In [27]:
lidf.columns[1]

'BU72'

In [31]:
df = pd.read_excel(fnames[2])
curdoc().clear()
#initial parameter for optimization
p0 = np.array([-7,-4,-4.5])
ligand_name = df.columns[1]
c, p_active = df['conc'].values, df[ligand_name].values

#optimize kd and energy difference to fit the data
res = scipy.optimize.least_squares(resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label=ligand_name+' [M]',
                          y_axis_label='active_probability',
                          x_axis_type='log')

p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
p.circle(c, p_active, size=7)

bokeh.io.show(p);
print(res.x)
print(10**res.x)

[-9.51908246 -8.50333157 -1.12017514]
[3.02633875e-10 3.13811193e-09 7.58271724e-02]


In [54]:
res.x

array([-9.6270621 , -7.09411868, -3.96001784])

In [14]:
curdoc().clear()
#initial parameter for optimization
p0 = np.array([-9,-4,-4])
c, p_active = df['conc'].values, df['fentanyl'].values/100

#optimize kd and energy difference to fit the data
res = scipy.optimize.least_squares(resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label='[fentanyl] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
p.circle(c, p_active, size=7, color=colors[i])

bokeh.io.show(p);
print(10**res.x)
print(res.x)

[1.50763146e-10 1.78055030e-07 7.27951523e-05]
[-9.82170481 -6.74944575 -4.13789754]


In [67]:
res.x

array([-9.82170484, -6.74944575, -4.1378976 ])

In [11]:
def partial_agonist_only_p_active_theory(c_agonist, log_Kd_active, log_Kd_inactive, beta_deltaE):
    "theoretical curve for mglur5 active probability"
    beta_deltaE = -4
    Kd_active = 10**(log_Kd_active)
    Kd_inactive = 10**(log_Kd_inactive)
    a = (1 + c_agonist/Kd_active)

    b = (1 + c_agonist/Kd_inactive)

    return a/(a+b*np.exp(-beta_deltaE))

def partial_resid(param, c, p_active):
    return p_active - partial_agonist_only_p_active_theory(c, *param)

In [12]:
curdoc().clear()
#initial parameter for optimization
p0 = np.array([-9,-7,-4.5])
c, p_active = df['conc'].values, df['lofentanyl'].values

#optimize kd and energy difference to fit the data
res = scipy.optimize.least_squares(partial_resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label='[lofentanyl] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

p.line(ligand_smooth, partial_agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
p.circle(c, p_active, size=7, color=colors[i])

bokeh.io.show(p);
print(10**res.x)
print(res.x)

[1.07989867e-13 1.62414430e-11 1.09647820e-04]
[-12.96661699 -10.78937539  -3.96      ]


In [25]:
curdoc().clear()
#initial parameter for optimization
p0 = np.array([-9,-7,-4])
c, p_active = df['conc'].values, df['PZM21'].values/100

#optimize kd and energy difference to fit the data
res = scipy.optimize.least_squares(partial_resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=400,
                         plot_width=600,
                         x_axis_label='[PZM21] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

p.line(ligand_smooth, partial_agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
p.circle(c, p_active, size=7, color=colors[i])

bokeh.io.show(p);
print(10**res.x)
print(res.x)

[4.24527530e-10 2.18645805e-08 1.00000000e-04]
[-9.37209414 -7.66025885 -4.        ]
